In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
file_name = '../data/demo-4.csv'
file_name_bmms = '../data/BMMS_overview.xlsx'
file_name_roads = '../data/_roads3.csv'
demo = pd.read_csv(file_name)
bmms = pd.read_excel(file_name_bmms)
roads = pd.read_csv(file_name_roads)

In [3]:
roads_25 = []
for road in roads['road'].unique():
    if 'N1' in road or 'N2' in road:
        length = roads[roads['road'] == road ]['chainage'].max()
        if length > 25:
            roads_25.append(road)
roads_25

['N1', 'N102', 'N104', 'N105', 'N106', 'N2', 'N204', 'N207', 'N208']

Roads required: ['N1', 'N102', 'N104', 'N105', 'N106', 'N2', 'N204', 'N207', 'N208']

In [4]:
def clean_file(roads, bmms, road_name):
    # Filter the 'roads' DataFrame for rows where the 'road' column is 'N1'
    n1_roads = roads[roads['road'] == road_name]
    # if road_name == 'N1':
    #     n1_roads = n1_roads[(n1_roads['lon'] <= 91.851) &
    #                      (n1_roads['lat'] >= 22.36)]
    print(n1_roads.head())
    # Generate an ID sequence starting from 1
    n1_roads['id'] = range(1, len(n1_roads) + 1)
    
    # Set model_type to 'link' for all rows
    n1_roads['model_type'] = 'link'
    
    # Generate 'name' as "Link" + id as string
    #n1_roads['name'] = ['Link ' + str(id) for id in n1_roads['id']]
    # n1_roads['name'] = 'Link'
    # Calculate 'length' as difference between this row's 'chainage' and the next row's 'chainage'
    # Shift(-1) moves the chainage up by one row to subtract, fillna(0) to handle the last item
    n1_roads['length'] = (n1_roads['chainage'].shift(-1) - n1_roads['chainage']).fillna(0)
    
    # Selecting the columns needed for the empty DataFrame
    n1_roads_final = n1_roads[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'length', 'chainage']]
    
    # Filter BMMS data for road 'N1'
    bmms_n1 = bmms[bmms['road'] == road_name].copy()
    # bmms_n1 = bmms_n1[(bmms_n1['lon'] <= 91.851) &
    #                  (bmms_n1['lat'] >= 22.363)]
    
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains(r"\(R\)", na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains("right", case=False, na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains("RIGHT", case=False, na=False)]
    bmms_n1 = bmms_n1[~bmms_n1['name'].str.contains(r"\( R \)", na=False)]
    
    # Set up for new entries
    bmms_n1['model_type'] = 'bridge'
    #bmms_n1['name'] = ['Bridge ' + str(i+1) for i in range(bmms_n1.shape[0])]
    bmms_n1['id'] = range(n1_roads_final['id'].max() + 1, n1_roads_final['id'].max() + 1 + bmms_n1.shape[0])
    bmms_n1['chainage'] = bmms_n1['km']  # Use 'km' as 'chainage'
    bmms_n1['length'] = bmms_n1['length'] / 1000
    
    # Select and rename columns to match the format of n1_roads_final_with_chainage
    bmms_n1_formatted = bmms_n1[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'chainage', 'length', 'condition']]
    
    # Combine the dataframes and sort by chainage
    combined_df = pd.concat([n1_roads_final, bmms_n1_formatted], ignore_index=True).sort_values(by='chainage')
    
    combined_df.iloc[0, 2] = 'sourcesink'
    combined_df.iloc[-1, combined_df.columns.get_loc('model_type')] = 'sourcesink'
    combined_df.reset_index(drop=True, inplace=True)
    count = 1
    for index, row in combined_df.iterrows():
        combined_df.iloc[index, 1] = count
        count += 1
    
    n1 = combined_df
    
    duplicates_df = bmms_n1[bmms_n1.duplicated('km', keep=False)]

    # Assuming duplicates_df is your DataFrame


    # Convert 'condition' to a numerical value for averaging
    condition_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
    n1['condition_num'] = n1['condition'].map(condition_mapping)
    
    #add together the links
    amount = 0
    condition_sum = 0
    rows_to_add = []  # List to accumulate rows
    last_row = None
    
    # Initialize the DataFrame
    n1_new = pd.DataFrame(columns=['road', "id", 'model_type', 'name', 'lon', 'lat', 'length', 'condition', 'condition_num'])
    for index, row in n1.iterrows():
        if last_row is not None:
            if row['model_type'] == 'bridge':
                amount += 1
                condition_sum += row['condition_num']
            if row['model_type'] != 'bridge' and last_row['model_type'] != 'bridge':
                rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                    'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                    'condition_num': row['condition_num']})
                amount = 0
                condition_sum = 0
            if row['model_type'] != 'bridge' and last_row['model_type'] == 'bridge':
                rows_to_add.append({'road': last_row['road'], 'id': last_row['id'], 'model_type': last_row['model_type'],
                                    'name': last_row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': last_row['length'],
                                    'condition': last_row['condition'], 'condition_num': condition_sum / amount})
                amount = 0
                condition_sum = 0
                rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                    'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                    'condition_num': row['condition_num']})
        else:
            rows_to_add.append({'road': row['road'], 'id': row['id'], 'model_type': row['model_type'],
                                'name': row['name'],'lon': row['lon'], 'lat':row['lat'], 'length': row['length'], 'condition': row['condition'],
                                'condition_num': row['condition_num']})
        last_row = row
    
    n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)

    for index, row in n1_new.iterrows():
        if pd.isna(row['condition_num']) == False:
            n1_new.iloc[index, 8] = math.ceil(n1_new.iloc[index, 8])
    condition_mapping = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E'}
    n1_new['condition'] = n1_new['condition_num'].map(condition_mapping)
    # n1_new
    # n1_new['id'] = range(1, len(n1_new) + 1)
    # for index, row in n1_new.iterrows():
    #     if row['model_type'] != 'bridge':
    #         n1_new.iloc[index, 3] = row['model_type'] + ' ' + str(row['id'])
    #     else:
    #         n1_new.iloc[index, 3] = n1_new.iloc[index, 3] + ' ' + str(row['id'])
    # print(n1_new)
    return n1_new

In [5]:
all_roads = []
for road in ['N1', 'N102', 'N104', 'N105', 'N106', 'N2', 'N204', 'N207', 'N208']:
    all_roads.append(clean_file(roads, bmms, road))
merged_dataframe = pd.concat(all_roads, ignore_index=True)
desired_order = ['road', 'id', 'model_type', 'condition', 'name', 'lat', 'lon', 'length']
merged_dataframe = merged_dataframe[desired_order]
count = 1
for index, row in merged_dataframe.iterrows():
    merged_dataframe.loc[index, 'id'] = str(count)
    count+=1
    if row.model_type == 'link':
        merged_dataframe.loc[index, 'name'] = 'link ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'source':
        merged_dataframe.loc[index, 'name'] = 'source ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'sink':
        merged_dataframe.loc[index, 'name'] = 'sink ' + merged_dataframe.loc[index, 'id']
    elif row.model_type == 'bridge':
        merged_dataframe.loc[index, 'name'] = merged_dataframe.loc[index, 'name'] + ' id ' + merged_dataframe.loc[index, 'id']


  road  chainage     lrp        lat        lon  gap       type  \
0   N1     0.000    LRPS  23.706028  90.443333  NaN     Others   
1   N1     0.814   LRPSa  23.702917  90.450417  NaN    Culvert   
2   N1     0.822   LRPSb  23.702778  90.450472  NaN  CrossRoad   
3   N1     1.000  LRP001  23.702139  90.451972  NaN     KmPost   
4   N1     2.000  LRP002  23.697889  90.460583  NaN     KmPost   

                                                name  
0  Start of Road  after Jatrabari Flyover infront...  
1                                        Box Culvert  
2                            Intersection with Z1101  
3                                    Km post missing  
4                                    Km post missing  


C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['id'] = range(1, len(n1_roads) + 1)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['model_type'] = 'link'
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

      road  chainage     lrp        lat        lon  gap     type  \
1356  N102     0.000    LRPS  23.478972  91.118194  NaN   Others   
1357  N102     0.329   LRPSa  23.481583  91.116777  NaN  Culvert   
1358  N102     0.995   LRPSb  23.486666  91.113361  NaN  Culvert   
1359  N102     1.000  LRP001  23.486722  91.113306  NaN   KmPost   
1360  N102     2.000  LRP002  23.493527  91.106917  NaN   KmPost   

                                 name  
1356  Start of road from N1 Mainamati  
1357                      Box Culvert  
1358                      Box Culvert  
1359                  Km post missing  
1360      B Baria 68 km Sylhet 248 km  
      road  chainage      lrp        lat        lon  gap       type  \
1587  N104     0.000     LRPS  23.009667  91.399416  NaN     Others   
1588  N104     0.445    LRPSa  23.009278  91.395250  NaN    Culvert   
1589  N104     1.000   LRP001  23.009306  91.389805  NaN     KmPost   
1590  N104     1.878  LRP001a  23.009528  91.381444  NaN  CrossRoad

C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['id'] = range(1, len(n1_roads) + 1)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

      road  chainage     lrp        lat        lon  gap     type  \
1827  N106     0.000    LRPS  22.393555  91.821583  NaN   Others   
1828  N106     0.480   LRPSa  22.397500  91.820027  NaN  Culvert   
1829  N106     0.648   LRPSb  22.399055  91.819805  NaN  Culvert   
1830  N106     0.822   LRPSc  22.400472  91.819360  NaN  Culvert   
1831  N106     1.000  LRP001  22.402056  91.819110  NaN   KmPost   

                                  name  
1827  Start of Road  from Oxygen  More  
1828                       Box culvert  
1829                       Box culvert  
1830                       Box culvert  
1831                   Km post missing  
     road  chainage      lrp        lat        lon  gap    type  \
2346   N2     0.000     LRPS  23.705917  90.521444  NaN  Others   
2347   N2     0.166   LRP001  23.706889  90.520472  NaN  KmPost   
2348   N2     1.166   LRP002  23.715139  90.516833  NaN  KmPost   
2349   N2     1.536  LRP002a  23.718277  90.515694   BS  Bridge   
2350   N2 

C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['id'] = range(1, len(n1_roads) + 1)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

      road  chainage      lrp        lat        lon  gap     type  \
3237  N204     0.000     LRPS  24.147916  91.346611  NaN   Others   
3238  N204     0.545    LRPSa  24.148361  91.351389  NaN  Culvert   
3239  N204     0.630   LRP001  24.147555  91.351361  NaN   KmPost   
3240  N204     0.806  LRP001a  24.146027  91.351306   BS   Bridge   
3241  N204     0.848  LRP001b  24.145722  91.351306   BE   Bridge   

                                      name  
3237  Road start from N2 at Jagannathganj.  
3238                           Box culvert  
3239                        Hobiganj 31 km  
3240                          Bridge start  
3241                            Bridge end  
      road  chainage     lrp        lat        lon  gap     type  \
3346  N207     0.000    LRPS  24.294861  91.510250  NaN   Others   
3347  N207     0.226   LRPSa  24.296416  91.512083  NaN  Culvert   
3348  N207     0.457   LRPSb  24.297583  91.513528   BS   Bridge   
3349  N207     0.479   LRPSc  24.297555  91

C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_new = pd.concat([n1_new, pd.DataFrame(rows_to_add)], ignore_index=True)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['id'] = range(1, len(n1_roads) + 1)
C:\Users\svenk\AppData\Local\Temp\ipykernel_14740\3266304506.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [6]:
merged_dataframe

,road,id,model_type,condition,name,lat,lon,length
0,N1,1,sourcesink,NaN,Start of Road after Jatrabari Flyover infront...,23.706028,90.443333,0.8140
1,N1,2,link,NaN,link 2,23.702917,90.450417,0.0080
2,N1,3,link,NaN,link 3,23.702778,90.450472,0.1780
3,N1,4,link,NaN,link 4,23.702139,90.451972,1.0000
4,N1,5,bridge,A,. id 5,23.697889,90.460583,0.0113
...,...,...,...,...,...,...,...,...
4709,N208,4710,link,NaN,link 4710,24.861528,91.881860,0.0850
4710,N208,4711,link,NaN,link 4711,24.861889,91.881027,1.0000
4711,N208,4712,link,NaN,link 4712,24.869499,91.876027,0.9000
4712,N208,4713,bridge,B,KADAM TALI BOX CULVERT id 4713,24.877555,91.875360,0.0056


In [7]:
demo = pd.read_csv('../data/demo-4.csv')
demo

,road,id,model_type,condition,name,lat,lon,length
0,N1,1000000,sourcesink,NaN,SoSi1,0,0,4
1,N1,1000001,link,NaN,NaN,1,1,1800
2,N1,1000002,bridge,A,NaN,2,2,200
3,N1,1000003,link,NaN,NaN,3,3,1500
4,N1,1000004,bridge,A,NaN,4,4,8
5,N1,1000005,link,NaN,NaN,5,5,2000
6,N1,1000006,intersection,NaN,NaN,6,6,20
7,N1,1000007,link,NaN,NaN,7,7,1400
8,N1,1000008,bridge,A,NaN,8,8,4
9,N1,1000009,link,NaN,NaN,9,9,2500


In [8]:
for index, row_ss in merged_dataframe[merged_dataframe['model_type'] == 'sourcesink'].iterrows():
    closest_lon = 100
    closest_lat = 100
    closest_diff = 100
    closest_id = None
    closest_row = None
    for index2, row in merged_dataframe.iterrows():
        if row.road != row_ss.road and row.model_type != 'bridge':
            diff_lat = abs(row_ss.lat-row.lat)
            diff_lon = abs(row_ss.lon-row.lon)
            diff = diff_lat + diff_lon
            if diff < closest_diff:
                closest_lon = row.lon 
                closest_lat = row.lat
                closest_id = row.id
                closest_diff = diff
                closest_row = row
    if closest_diff < 0.06:
        merged_dataframe.loc[merged_dataframe['id'] == row_ss.id, 'model_type'] = 'intersection'
        merged_dataframe.loc[merged_dataframe['id'] == row_ss.id, 'name'] = f"intersection of {row_ss.road} with {closest_row.road}"
        merged_dataframe.loc[merged_dataframe['id'] == row_ss.id, 'intersects_with'] = closest_row.id
        merged_dataframe.loc[merged_dataframe['id'] == closest_id, 'model_type'] = 'intersection'
        merged_dataframe.loc[merged_dataframe['id'] == closest_id, 'name'] = f"intersection of {closest_row.road} with {row_ss.road}"
        merged_dataframe.loc[merged_dataframe['id'] == closest_id, 'intersects_with'] = row_ss.id
        print('\n Correct!\ndifference is:', closest_diff)
        print(f"intersection of {row_ss.road} with {closest_row.road}")
    else:
        print('\n NOPE!\ndifference is:', closest_diff)
        print(f"intersection of {row_ss.road} with {closest_row.road}")
    


 NOPE!
difference is: 0.0779996999999959
intersection of N1 with N2

 NOPE!
difference is: 1.8818604000000008
intersection of N1 with N106

 Correct!
difference is: 2.779999999802385e-05
intersection of N102 with N1

 Correct!
difference is: 0.0004445000000039556
intersection of N102 with N2

 Correct!
difference is: 0.013667000000005203
intersection of N104 with N1

 NOPE!
difference is: 0.45319439999999744
intersection of N104 with N1

 Correct!
difference is: 2.779999999802385e-05
intersection of N105 with N1

 NOPE!
difference is: 0.36861039999999434
intersection of N105 with N1

 Correct!
difference is: 0.058194700000012034
intersection of N106 with N1

 NOPE!
difference is: 0.5611947999999991
intersection of N106 with N1

 Correct!
difference is: 0.0002499000000000251
intersection of N2 with N1

 NOPE!
difference is: 0.42177849999998784
intersection of N2 with N208

 Correct!
difference is: 0.00022219999999251172
intersection of N204 with N2

 Correct!
difference is: 0.000555599

In [9]:
merged_dataframe[merged_dataframe['model_type'] == 'intersection']

,road,id,model_type,condition,name,lat,lon,length,intersects_with
19,N1,20,intersection,NaN,intersection of N1 with N2,23.706083,90.521527,0.740,2801
34,N1,35,intersection,NaN,intersection of N1 with N105,23.690416,90.546583,0.588,2353
243,N1,244,intersection,NaN,intersection of N1 with N102,23.478972,91.118166,0.547,1885
448,N1,449,intersection,NaN,intersection of N1 with N104,22.996389,91.399805,0.085,2192
757,N1,758,intersection,NaN,intersection of N1 with N106,22.364611,91.850833,0.169,2502
1884,N102,1885,intersection,NaN,intersection of N102 with N1,23.478972,91.118194,0.329,244
2190,N102,2191,intersection,NaN,intersection of N102 with N2,24.050611,91.114667,0.000,3126
2191,N104,2192,intersection,NaN,intersection of N104 with N1,23.009667,91.399416,0.445,449
2352,N105,2353,intersection,NaN,intersection of N105 with N1,23.690416,90.546611,1.000,35
2501,N106,2502,intersection,NaN,intersection of N106 with N1,22.393555,91.821583,0.480,758


In [10]:
merged_dataframe.to_csv('../data/merged_data.csv', index=False)

In [27]:
merged_dataframe

,road,id,model_type,condition,name,lat,lon,length,intersects_with
0,N1,1,sourcesink,NaN,Start of Road after Jatrabari Flyover infront...,23.706028,90.443333,0.8140,nan
1,N1,2,link,NaN,link 2,23.702917,90.450417,0.0080,nan
2,N1,3,link,NaN,link 3,23.702778,90.450472,0.1780,nan
3,N1,4,link,NaN,link 4,23.702139,90.451972,1.0000,nan
4,N1,5,bridge,A,. id 5,23.697889,90.460583,0.0113,nan
...,...,...,...,...,...,...,...,...,...
4709,N208,4710,link,NaN,link 4710,24.861528,91.881860,0.0850,nan
4710,N208,4711,link,NaN,link 4711,24.861889,91.881027,1.0000,nan
4711,N208,4712,link,NaN,link 4712,24.869499,91.876027,0.9000,nan
4712,N208,4713,bridge,B,KADAM TALI BOX CULVERT id 4713,24.877555,91.875360,0.0056,nan
